
### Este conjunto de dados se trate de empréstimos financeiros
#### **O objetivo deste notebook é prever se uma transação é uma possível fraude ou não**

Entre as colunas deste dataset temos: contrato, idade, sexo , estado civil, UF do Cliente, data de Contratação, Valor do empréstimo, valor do empréstimo com juros, escolaridade, quantidade de parcelas pagas, quantidade de parcelas pagas em dia, quantidade de parcelas pagas em atraso, quantidade de renegociações, se o cliente possui patrimônio, valor do patrimônio, saldo devedor, total pago, valor da renda, percentual de juros, quantidade de parcelas com atraso, quantidade de dias de atraso, prazo restante e prazo de empréstimo.

A coluna alvo é: **Possivel_Fraude**

Este é um problema de **Classificação**

Os passos a serem dados neste projeto são:

* Analise exploratória
* Tratamento dos dados
* Análise estatística
* Engenharia de atributos
* Visualização com gráficos
* OneHotEncoding
* Balanceamento da variável **Possivel_Fraude**
* Normalização e Padronização dos dados
* Criação de modelos de ML
* Treinamento dos modelos
* Ajuste de hyperparâmetros
* Análise e predição dos modelos
* Colocar o modelo em produção

In [661]:
#importando as bibliotecas
import warnings
import pandas as pd
import numpy as np
import plotly.express as px
import time
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
warnings.filterwarnings("ignore")


In [662]:
#importando os dados
df = pd.read_csv(r"dataset/dados_coletados80k.csv")

## Análise Exploratória dos dados

In [663]:
#visualizando o dataset
df.sample(4)

,Contrato,Idade,Sexo,Valor_Renda,UF_Cliente,Perc_Juros,Prazo_Emprestimo,Data_Contratacao,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,Estado_Civil,Escolaridade,Possui_Patrimonio,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago,Possivel_Fraude
41617,321985145715,22,M,1200.00,MG,18.00,100,2021-07-07,84,25000.00,28750.00,26,20,5,1,SOLTEIRO(A),NaN,N,0.00,2,41.00,22039.01,8058.27,Nao
29794,322000148715,33,F,2500.00,PI,22.00,200,2021-08-31,179,60000.00,69000.00,1,1,0,0,SOLTEIRO(A),Nenhum,N,0.00,25,741.00,74124.68,381.40,Sim
35063,322020400715,26,F,5000.00,RS,16.00,120,2021-11-26,125,120000.00,138000.00,3,2,0,1,SOLTEIRO(A),Nenhum,N,0.00,20,589.00,138929.50,2009.88,Sim
67131,321960510715,25,F,1250.00,RJ,21.00,55,2021-03-08,25,10000.00,11500.00,2,1,0,0,SOLTEIRO(A),NaN,N,0.00,30,894.00,11923.83,447.28,Sim


In [664]:
df.shape

(80143, 24)

Dataset contém 80143 linhas e 24 colunas

In [665]:
#visualizar algumas metricas do df
df.describe()

,Contrato,Idade,Valor_Renda,Perc_Juros,Prazo_Emprestimo,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago
count,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,54348.00,80143.00,80143.00
mean,322023695416.17,38.82,29308.62,19.49,102.33,93.28,72847.78,83774.94,10.71,5.24,2.34,1.30,1.45,10.65,475.26,78622.32,9687.98
std,44697010.94,12.53,2748166.27,3.56,62.32,70.64,87407.93,100519.12,9.31,6.21,3.97,1.27,355.10,10.43,269.79,102318.95,19090.62
min,321944855715.00,4.00,0.00,4.49,14.00,-18.00,3000.00,3450.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,0.00
25%,321984967215.00,29.00,2050.00,18.00,55.00,36.00,15000.00,17250.00,2.00,1.00,0.00,0.00,0.00,0.00,284.00,12122.76,1052.16
50%,322024156715.00,37.00,3000.00,20.00,80.00,78.00,40000.00,46000.00,8.00,2.00,1.00,1.00,0.00,9.00,468.00,35489.36,3951.39
75%,322062662215.00,46.00,5000.00,21.00,170.00,156.00,100000.00,115000.00,18.00,7.00,3.00,2.00,0.00,20.00,708.00,117783.25,10886.24
max,322099706715.00,103.00,765000000.00,28.00,240.00,227.00,500000.00,575000.00,66.00,66.00,31.00,38.00,100000.00,34.00,1014.00,626261.27,465821.39


Podemos tirar alguns insights como:

* Desvio padrão das variáveis alto, precisaremos usar ou a normalização ou a padronização dos dados antes de usar um modelo de machine learning

* Média de idade de 38/39 anos
* Idade mínima de 4 anos (possível erro) e Idade máxima de 103 anos

* Renda média de 29.308 , mínima de 0 e máxima 765000000.00 (possível outlier)

* Média de 10 parcelas pagas, 5 pagas em dia e 2 pagas com atraso
* Média de 10 parcelas com atraso

In [666]:
#visualizar metricas das colunas nao numericas
df.describe(exclude="number")

,Sexo,UF_Cliente,Data_Contratacao,Estado_Civil,Escolaridade,Possui_Patrimonio,Possivel_Fraude
count,80143,80143,80143,80143,22845,80143,80143
unique,2,27,751,8,5,2,2
top,M,SP,2021-11-30,SOLTEIRO(A),Nenhum,N,Sim
freq,48500,21393,374,45205,21168,79785,44511


In [667]:
#erificando a media do Valor_Renda para cada nivel de escolaridade e sexo
df.groupby(["Escolaridade", "Sexo"])["Valor_Renda"].mean().to_frame()

Valor_Renda
Escolaridade                         Sexo             
Ensino Fundamental                   F         2619.96
                                     M         4533.91
Ensino Médio                         F         4885.93
                                     M         5810.71
Ensino Superior                      F         9990.37
                                     M        40138.40
Nenhum                               F         8238.12
                                     M       101368.94
Pós Graduação / Mestrado / Doutorado F        10115.88
                                     M        10673.08

In [668]:
#verificando a distribuicao da coluna UF_Cliente
df["UF_Cliente"].value_counts(normalize=True)

UF_Cliente
SP   0.27
MG   0.17
BA   0.08
PR   0.07
GO   0.05
MA   0.04
RS   0.04
PA   0.04
SC   0.03
PE   0.03
RJ   0.03
MS   0.03
CE   0.03
PB   0.02
MT   0.01
AL   0.01
RN   0.01
PI   0.01
DF   0.01
SE   0.01
ES   0.00
TO   0.00
AM   0.00
RO   0.00
AC   0.00
AP   0.00
RR   0.00
Name: proportion, dtype: float64

In [669]:
#verificando o numero de valores unicos de cada coluna
df.nunique()

Contrato                            80143
Idade                                  86
Sexo                                    2
Valor_Renda                          4747
UF_Cliente                             27
Perc_Juros                             53
Prazo_Emprestimo                       60
Data_Contratacao                      751
Prazo_Restante                         93
VL_Emprestimo                          73
VL_Emprestimo_ComJuros                 73
QT_Total_Parcelas_Pagas                50
QT_Total_Parcelas_Pagas_EmDia          48
QT_Total_Parcelas_Pagas_EmAtraso       32
Qt_Renegociacao                        15
Estado_Civil                            8
Escolaridade                            5
Possui_Patrimonio                       2
VL_Patrimonio                           4
QT_Parcelas_Atraso                     35
QT_Dias_Atraso                         38
Saldo_Devedor                       64166
Total_Pago                          54074
Possivel_Fraude                   

In [670]:
df.groupby(["Escolaridade"]).size()

Escolaridade
Ensino Fundamental                        126
Ensino Médio                              902
Ensino Superior                           600
Nenhum                                  21168
Pós Graduação / Mestrado / Doutorado       49
dtype: int64

In [671]:
df.groupby(["Estado_Civil"]).size()

Estado_Civil
CASADO (A)                23576
DIVORCIADO                 3783
NENHUM                        4
OUTRO                      5912
SEPARADO JUDICIALMENTE        4
SOLTEIRO(A)               45205
UNIÃO ESTAVEL               501
VIÚVO(A)                   1158
dtype: int64

In [672]:
df.groupby(["Possui_Patrimonio"]).size()

Possui_Patrimonio
N    79785
S      358
dtype: int64

In [673]:
df.groupby(["VL_Patrimonio"]).size()

VL_Patrimonio
0.00         80135
1000.00          6
10000.00         1
100000.00        1
dtype: int64

In [674]:
df.groupby(["QT_Dias_Atraso"]).size().sort_values(ascending=False)

QT_Dias_Atraso
11.00      3626
741.00     2180
376.00     2177
649.00     2036
406.00     2024
344.00     1986
435.00     1980
468.00     1977
251.00     1957
284.00     1956
771.00     1952
708.00     1936
617.00     1911
498.00     1873
559.00     1848
589.00     1840
680.00     1836
315.00     1833
529.00     1750
802.00     1731
41.00      1572
833.00     1471
225.00     1330
862.00     1174
71.00      1115
894.00     1061
103.00     1028
194.00     1027
924.00      988
953.00      962
162.00      914
133.00      860
981.00      427
437.00        3
736.00        3
1014.00       2
764.00        1
705.00        1
dtype: int64

In [675]:
df.groupby(["QT_Parcelas_Atraso"]).size().sort_values(ascending=False)

QT_Parcelas_Atraso
0     25795
1      4100
13     2146
25     2031
2      1992
19     1979
14     1979
22     1975
24     1967
16     1964
15     1951
12     1943
17     1860
26     1839
21     1826
10     1821
9      1793
18     1780
23     1778
20     1763
11     1740
27     1643
3      1600
28     1386
8      1297
29     1151
4      1139
7      1020
5       988
30      943
31      908
6       886
32      812
33      346
34        2
dtype: int64

Alguns insights:

* 27 tipos de dados diferentes na coluna *UF_Cliente*, este dataset contém clientes de todos os 26 estados no Brasil + DF
* A maior distribuição de dados da coluna *UF_Cliente*, está em São Paulo seguido por Minas Gerais
* 8 tipos de estado civil no dataset
* Homens tendem a ganhar mais do que as mulheres, e normalmente quanto maior o grau de escolaridade, maior o salário
* A maioria das pessoas no dataset não possui escolaridade e apenas 49 tem Pós Graduação / Mestrado / Doutorado
* A maioria das pessoas no dataset é SOLTEIRO(A)
* A maioria das pessoas no dataset não possui Patrimonio
* A coluna VL_Patrimonio indica que a coleta destes dados provavelmente está equivocada, devido ao fato de que quase todos os valores de patrimonio são 0 e apenas algunas poucos de 1.000, 10.000 e 100.00 (iremos retirar esta coluna)
* O número de dias atraso com o maior número de pessoas é de 11 dias
* A maioria das pessoas não tem parcelas em atraso

In [676]:
#visualizacao geral dos dados
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Contrato                          80143 non-null  int64  
 1   Idade                             80143 non-null  int64  
 2   Sexo                              80143 non-null  object 
 3   Valor_Renda                       80143 non-null  float64
 4   UF_Cliente                        80143 non-null  object 
 5   Perc_Juros                        80143 non-null  float64
 6   Prazo_Emprestimo                  80143 non-null  int64  
 7   Data_Contratacao                  80143 non-null  object 
 8   Prazo_Restante                    80143 non-null  int64  
 9   VL_Emprestimo                     80143 non-null  float64
 10  VL_Emprestimo_ComJuros            80143 non-null  float64
 11  QT_Total_Parcelas_Pagas           80143 non-null  int64  
 12  QT_T

In [677]:
#verificando o numero de valores nulos
df.isnull().sum()

Contrato                                0
Idade                                   0
Sexo                                    0
Valor_Renda                             0
UF_Cliente                              0
Perc_Juros                              0
Prazo_Emprestimo                        0
Data_Contratacao                        0
Prazo_Restante                          0
VL_Emprestimo                           0
VL_Emprestimo_ComJuros                  0
QT_Total_Parcelas_Pagas                 0
QT_Total_Parcelas_Pagas_EmDia           0
QT_Total_Parcelas_Pagas_EmAtraso        0
Qt_Renegociacao                         0
Estado_Civil                            0
Escolaridade                        57298
Possui_Patrimonio                       0
VL_Patrimonio                           0
QT_Parcelas_Atraso                      0
QT_Dias_Atraso                      25795
Saldo_Devedor                           0
Total_Pago                              0
Possivel_Fraude                   

As colunas  *Escolaridade* e *QT_Dias_Atraso* possuem valores ausentes                   

In [678]:
#verificando os tipos dos dados
df.dtypes

Contrato                              int64
Idade                                 int64
Sexo                                 object
Valor_Renda                         float64
UF_Cliente                           object
Perc_Juros                          float64
Prazo_Emprestimo                      int64
Data_Contratacao                     object
Prazo_Restante                        int64
VL_Emprestimo                       float64
VL_Emprestimo_ComJuros              float64
QT_Total_Parcelas_Pagas               int64
QT_Total_Parcelas_Pagas_EmDia         int64
QT_Total_Parcelas_Pagas_EmAtraso      int64
Qt_Renegociacao                       int64
Estado_Civil                         object
Escolaridade                         object
Possui_Patrimonio                    object
VL_Patrimonio                       float64
QT_Parcelas_Atraso                    int64
QT_Dias_Atraso                      float64
Saldo_Devedor                       float64
Total_Pago                      

In [679]:
#observando o inicio e fim dos dados da coluna Data_Contratacao
inicio = pd.to_datetime(df["Data_Contratacao"]).dt.date.min() 
fim = pd.to_datetime(df["Data_Contratacao"]).dt.date.max()

print("Período dos dados de:", inicio, "até", fim)

Período dos dados de: 2020-11-25 até 2023-02-22


O dataset tem dados de 25/11/2020 até 22/02/2023

## Tratamento dos dados e Engenharia de atributos

In [680]:
#otimizando a coluna Data_Contratacao para datetime64[ns]                  
df["Data_Contratacao"] = pd.to_datetime(df["Data_Contratacao"])

In [681]:
#otimizando colunas categoricas
colunas_categoricas = ["Sexo", "Estado_Civil", "Escolaridade", "Possui_Patrimonio", "Possivel_Fraude"]

for coluna in colunas_categoricas:
	df[coluna] = df[coluna].astype("category")

In [682]:
#verificando se a coluna escolaridade esta ordenada
df["Escolaridade"].cat.ordered

False

In [683]:
#verificando os valores da coluna escolaridade
df["Escolaridade"].value_counts()

Escolaridade
Nenhum                                  21168
Ensino Médio                              902
Ensino Superior                           600
Ensino Fundamental                        126
Pós Graduação / Mestrado / Doutorado       49
Name: count, dtype: int64

In [684]:
#ordenando a coluna escolaridade
escolaridade = ["Nenhum", "Ensino Médio", "Ensino Superior", "Ensino Fundamental", "Pós Graduação / Mestrado / Doutorado"]
df["Escolaridade"] = df["Escolaridade"].cat.set_categories(escolaridade, ordered = True)

print(df["Escolaridade"].cat.ordered)
print(df["Escolaridade"].unique())

True
[NaN, 'Nenhum', 'Ensino Médio', 'Ensino Superior', 'Ensino Fundamental', 'Pós Graduação / Mestrado / Doutorado']
Categories (5, object): ['Nenhum' < 'Ensino Médio' < 'Ensino Superior' < 'Ensino Fundamental' < 'Pós Graduação / Mestrado / Doutorado']


In [685]:
#otimizando colunas numericas
colunas_float = df.select_dtypes(include="float64").columns
colunas_int = df.select_dtypes(include="int64").columns

df[colunas_float] = df[colunas_float].apply(pd.to_numeric, downcast="float")
df[colunas_int] = df[colunas_int].apply(pd.to_numeric, downcast="integer")

In [686]:
#verificando a mudanco de tamanho do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Contrato                          80143 non-null  int64         
 1   Idade                             80143 non-null  int8          
 2   Sexo                              80143 non-null  category      
 3   Valor_Renda                       80143 non-null  float64       
 4   UF_Cliente                        80143 non-null  object        
 5   Perc_Juros                        80143 non-null  float32       
 6   Prazo_Emprestimo                  80143 non-null  int16         
 7   Data_Contratacao                  80143 non-null  datetime64[ns]
 8   Prazo_Restante                    80143 non-null  int16         
 9   VL_Emprestimo                     80143 non-null  float32       
 10  VL_Emprestimo_ComJuros            80143 non-nu

Reduzimos um pouco o tamanho do dataframe de 14.7 MB para 6.3 MB, e colocamos ordem na coluna *Escolaridade*

In [687]:
#removendo linhas duplicadas (neste dataset nao tem)
df = df.drop_duplicates()
df.shape

(80143, 24)

In [688]:
#renomeando o estado civil "NENHUM" para "OUTRO", renomeando "Uniao Estavel" para "Casado(A)", e renomeando "SEPARADO JUDICIALMENTE" para "DIVORCIADO"
df["Estado_Civil"] = df["Estado_Civil"].replace(["NENHUM"], "OUTRO")
df["Estado_Civil"] = df["Estado_Civil"].replace(["UNIÃO ESTAVEL" , "CASADO (A)"], "CASADO(A)")
df["Estado_Civil"] = df["Estado_Civil"].replace(["SEPARADO JUDICIALMENTE"], "DIVORCIADO")

df.groupby(["Estado_Civil"]).size()

Estado_Civil
CASADO(A)      24077
DIVORCIADO      3787
OUTRO           5916
SOLTEIRO(A)    45205
VIÚVO(A)        1158
dtype: int64

In [689]:
#verificando novamente o numero de valores unicos de cada coluna
df.nunique()

Contrato                            80143
Idade                                  86
Sexo                                    2
Valor_Renda                          4747
UF_Cliente                             27
Perc_Juros                             53
Prazo_Emprestimo                       60
Data_Contratacao                      751
Prazo_Restante                         93
VL_Emprestimo                          73
VL_Emprestimo_ComJuros                 73
QT_Total_Parcelas_Pagas                50
QT_Total_Parcelas_Pagas_EmDia          48
QT_Total_Parcelas_Pagas_EmAtraso       32
Qt_Renegociacao                        15
Estado_Civil                            5
Escolaridade                            5
Possui_Patrimonio                       2
VL_Patrimonio                           4
QT_Parcelas_Atraso                     35
QT_Dias_Atraso                         38
Saldo_Devedor                       64166
Total_Pago                          54074
Possivel_Fraude                   

In [690]:
df[["Idade", "Valor_Renda", "Total_Pago", "QT_Dias_Atraso", "Saldo_Devedor", "Prazo_Emprestimo", "Prazo_Restante"]].describe()

,Idade,Valor_Renda,Total_Pago,QT_Dias_Atraso,Saldo_Devedor,Prazo_Emprestimo,Prazo_Restante
count,80143.00,80143.00,80143.00,54348.00,80143.00,80143.00,80143.00
mean,38.82,29308.62,9687.98,475.26,78622.32,102.33,93.28
std,12.53,2748166.27,19090.62,269.79,102318.95,62.32,70.64
min,4.00,0.00,0.00,11.00,0.00,14.00,-18.00
25%,29.00,2050.00,1052.16,284.00,12122.76,55.00,36.00
50%,37.00,3000.00,3951.39,468.00,35489.36,80.00,78.00
75%,46.00,5000.00,10886.24,708.00,117783.25,170.00,156.00
max,103.00,765000000.00,465821.39,1014.00,626261.27,240.00,227.00


In [691]:
#criando intervalos de dados para colunas com muitos dados distintos (Valor Renda, Idade, Total Pago, Dias de atraso, Saldo Devedor, Prazo Emprestimo(meses), Prazo Restante(meses))
faixas = [0, 20, 30, 40, 50, 60, 70, 110]
categorias = ["0-20 Anos", "21-30 Anos", "31-40 Anos", "41-50 Anos", "51-60 Anos", "61-70 Anos", "71-110 Anos"]
df["Faixa_Etaria"] = pd.cut(df["Idade"], bins=faixas, labels=categorias)

#verificando os intervalos
#df.groupby(["Faixa_Etaria", "Idade"]).size()


In [692]:
#ordenando a coluna Faixa_Etaria
categorias = ["0-20 Anos", "21-30 Anos", "31-40 Anos", "41-50 Anos", "51-60 Anos", "61-70 Anos", "71-110 Anos"]
df["Faixa_Etaria"] = df["Faixa_Etaria"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Etaria"].cat.ordered)
print(df["Faixa_Etaria"].unique())

True
['41-50 Anos', '31-40 Anos', '21-30 Anos', '0-20 Anos', '51-60 Anos', '61-70 Anos', '71-110 Anos']
Categories (7, object): ['0-20 Anos' < '21-30 Anos' < '31-40 Anos' < '41-50 Anos' < '51-60 Anos' < '61-70 Anos' < '71-110 Anos']


In [693]:
#faixa salarial min: 0 e max: 765.000.000
faixas = [-1, 1000, 3000, 5000, 10000, 20000, 50000, 100000, 500000, 1000000, 5000000, 999000000]
categorias = ["Até 1000", "1k - 3k", "3k - 5k", "5k - 10k", "10k - 20k", "20k - 50k", "50k - 100k", "100k - 500k", "500k - 1kk", "1kk - 5kk", "Mais de 5kk"]
df["Faixa_Salarial"] = pd.cut(df["Valor_Renda"], bins=faixas, labels=categorias)

#verificando os intervalos
df.groupby(["Faixa_Salarial"]).size()

Faixa_Salarial
Até 1000         255
1k - 3k        41119
3k - 5k        21105
5k - 10k       11680
10k - 20k       3505
20k - 50k       1574
50k - 100k       354
100k - 500k      390
500k - 1kk        46
1kk - 5kk         60
Mais de 5kk       55
dtype: int64

In [694]:
#ordenando a coluna Faixa_Salarial
categorias = ["Até 1000", "1k - 3k", "3k - 5k", "5k - 10k", "10k - 20k", "20k - 50k", "50k - 100k", "100k - 500k", "500k - 1kk", "1kk - 5kk", "Mais de 5kk"]
df["Faixa_Salarial"] = df["Faixa_Salarial"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Salarial"].cat.ordered)
print(df["Faixa_Salarial"].unique())

True
['3k - 5k', '1k - 3k', '5k - 10k', '10k - 20k', '20k - 50k', ..., '500k - 1kk', '50k - 100k', 'Mais de 5kk', '1kk - 5kk', '100k - 500k']
Length: 11
Categories (11, object): ['Até 1000' < '1k - 3k' < '3k - 5k' < '5k - 10k' ... '100k - 500k' < '500k - 1kk' < '1kk - 5kk' < 'Mais de 5kk']


In [695]:
#verificando media e mediana da coluna "QT_Dias_Atraso"
print("Mediana", df["QT_Dias_Atraso"].median())
print("Media", df["QT_Dias_Atraso"].mean())

Mediana 468.0
Media 475.2605


In [696]:
#verificamos antes que a coluna "QT_Dias_Atraso" tem 25795 valores faltando, vamos substituir eles com a mediana 
df["QT_Dias_Atraso"] = df["QT_Dias_Atraso"].fillna((df["QT_Dias_Atraso"].median()))
df[["QT_Dias_Atraso"]].isnull().sum()

QT_Dias_Atraso    0
dtype: int64

In [697]:
#"QT_Dias_Atraso" tem valor min: 11 e max 1014
faixas = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 366, 9999]
categorias = ["Até 30 dias", "De 30 - 60 dias", "De 60 - 90 dias", "De 90 - 120 dias", "De 120 - 150 dias", "De 150 - 180 dias", "De 180 - 210 dias", "De 210 - 240 dias", "De 240 - 270 dias", "De 270 -  300 dias", "De 300 - 330 dias", "De 330 - 1 ano", "Mais de 1 ano"]
df["Faixa_Dias_Atraso"] = pd.cut(df["QT_Dias_Atraso"], bins=faixas, labels=categorias)

#verificando os intervalos
#df.groupby(["Faixa_Dias_Atraso"]).size()
#df.groupby(["Faixa_Dias_Atraso", "QT_Dias_Atraso"]).size()

In [698]:
#ordenando a coluna Faixa_Dias_Atraso
categorias = ["Até 30 dias", "De 30 - 60 dias", "De 60 - 90 dias", "De 90 - 120 dias", "De 120 - 150 dias", "De 150 - 180 dias", "De 180 - 210 dias", "De 210 - 240 dias", "De 240 - 270 dias", "De 270 -  300 dias", "De 300 - 330 dias", "De 330 - 1 ano", "Mais de 1 ano"]
df["Faixa_Dias_Atraso"] = df["Faixa_Dias_Atraso"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Dias_Atraso"].cat.ordered)
print(df["Faixa_Dias_Atraso"].unique())

True
['De 270 -  300 dias', 'Mais de 1 ano', 'De 30 - 60 dias', 'De 150 - 180 dias', 'De 240 - 270 dias', ..., 'De 60 - 90 dias', 'De 300 - 330 dias', 'De 210 - 240 dias', 'De 180 - 210 dias', 'De 120 - 150 dias']
Length: 13
Categories (13, object): ['Até 30 dias' < 'De 30 - 60 dias' < 'De 60 - 90 dias' < 'De 90 - 120 dias' ... 'De 270 -  300 dias' < 'De 300 - 330 dias' < 'De 330 - 1 ano' < 'Mais de 1 ano']


In [699]:
#Total Pago min: 0 e max: 465.821,39
faixas = [-1, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 250000, 400000, 999999]
categorias = ["Até 1k", "De 1k - 2k", "De 2k - 5k", "De 5k - 10k", "De 10k - 20k", "De 20k - 50k", "De 50k - 100k", "De 100k - 250k", "De 250k - 400k",  "Mais de 400k"]
df["Faixa_Total_Pago"] = pd.cut(df["Total_Pago"], bins=faixas, labels=categorias)

#verificando os intervalos
df.groupby(["Faixa_Total_Pago"]).size()


Faixa_Total_Pago
Até 1k            19571
De 1k - 2k         9968
De 2k - 5k        14883
De 5k - 10k       13858
De 10k - 20k      11950
De 20k - 50k       7879
De 50k - 100k      1525
De 100k - 250k      421
De 250k - 400k       82
Mais de 400k          6
dtype: int64

In [700]:
#ordenando a coluna Faixa_Total_Pago
categorias = ["Até 1k", "De 1k - 2k", "De 2k - 5k", "De 5k - 10k", "De 10k - 20k", "De 20k - 50k", "De 50k - 100k", "De 100k - 250k", "De 250k - 400k",  "Mais de 400k"]
df["Faixa_Total_Pago"] = df["Faixa_Total_Pago"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Total_Pago"].cat.ordered)
print(df["Faixa_Total_Pago"].unique())

True
['De 1k - 2k', 'De 20k - 50k', 'Até 1k', 'De 5k - 10k', 'De 2k - 5k', 'De 10k - 20k', 'De 50k - 100k', 'De 100k - 250k', 'De 250k - 400k', 'Mais de 400k']
Categories (10, object): ['Até 1k' < 'De 1k - 2k' < 'De 2k - 5k' < 'De 5k - 10k' ... 'De 50k - 100k' < 'De 100k - 250k' < 'De 250k - 400k' < 'Mais de 400k']


In [701]:
#Saldo Devedor min: 0 e max: 626.261,27
faixas = [-1, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 250000, 500000, 999999]
categorias = ["Até 1k", "De 1k - 2k", "De 2k - 5k", "De 5k - 10k", "De 10k - 20k", "De 20k - 50k", "De 50k - 100k", "De 100k - 250k", "De 250k - 500k",  "Mais de 500k"]
df["Faixa_Saldo_Devedor"] = pd.cut(df["Saldo_Devedor"], bins=faixas, labels=categorias)

#verificando os intervalos
df.groupby(["Faixa_Saldo_Devedor"]).size()

Faixa_Saldo_Devedor
Até 1k             3636
De 1k - 2k          857
De 2k - 5k         4242
De 5k - 10k        8347
De 10k - 20k      10782
De 20k - 50k      19807
De 50k - 100k      9532
De 100k - 250k    17433
De 250k - 500k     4871
Mais de 500k        636
dtype: int64

In [702]:
#ordenando a coluna Faixa_Saldo_Devedor
categorias = ["Até 1k", "De 1k - 2k", "De 2k - 5k", "De 5k - 10k", "De 10k - 20k", "De 20k - 50k", "De 50k - 100k", "De 100k - 250k", "De 250k - 500k",  "Mais de 500k"]
df["Faixa_Saldo_Devedor"] = df["Faixa_Saldo_Devedor"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Saldo_Devedor"].cat.ordered)
print(df["Faixa_Saldo_Devedor"].unique())

True
['De 100k - 250k', 'De 10k - 20k', 'De 50k - 100k', 'De 20k - 50k', 'De 2k - 5k', 'De 250k - 500k', 'De 5k - 10k', 'De 1k - 2k', 'Até 1k', 'Mais de 500k']
Categories (10, object): ['Até 1k' < 'De 1k - 2k' < 'De 2k - 5k' < 'De 5k - 10k' ... 'De 50k - 100k' < 'De 100k - 250k' < 'De 250k - 500k' < 'Mais de 500k']


In [703]:
#Prazo Emprestimo(meses) min: 14 e max: 240.00
faixas = [-1, 60, 120, 200, 720]
categorias = ["Até 60 Meses", "De 61 até 120 Meses", "De 121 até 200 Meses", "Acima de 200 Meses"]
df["Faixa_Prazo_Emprestimo"] = pd.cut(df["Prazo_Emprestimo"], bins=faixas, labels=categorias)

#verificando os intervalos
df.groupby(["Faixa_Prazo_Emprestimo"]).size()

Faixa_Prazo_Emprestimo
Até 60 Meses            31764
De 61 até 120 Meses     25123
De 121 até 200 Meses    21455
Acima de 200 Meses       1801
dtype: int64

In [704]:
#ordenando a coluna Faixa_Prazo_Emprestimo
categorias = ["Até 60 Meses", "De 61 até 120 Meses", "De 121 até 200 Meses", "Acima de 200 Meses"]
df["Faixa_Prazo_Emprestimo"] = df["Faixa_Prazo_Emprestimo"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Prazo_Emprestimo"].cat.ordered)
print(df["Faixa_Prazo_Emprestimo"].unique())

True
['De 121 até 200 Meses', 'Até 60 Meses', 'De 61 até 120 Meses', 'Acima de 200 Meses']
Categories (4, object): ['Até 60 Meses' < 'De 61 até 120 Meses' < 'De 121 até 200 Meses' < 'Acima de 200 Meses']


In [705]:
#Prazo Restante(meses) min: -18.00 e max: 227
faixas = [-20, 60, 120, 200, 500]
categorias = ["Até 60 Meses", "De 61 até 120 Meses", "De 121 até 200 Meses", "Acima de 200 Meses"]
df["Faixa_Prazo_Restante"] = pd.cut(df["Prazo_Restante"], bins=faixas, labels=categorias)

#verificando os intervalos
df.groupby(["Faixa_Prazo_Restante"]).size()

Faixa_Prazo_Restante
Até 60 Meses            31032
De 61 até 120 Meses     24752
De 121 até 200 Meses    15266
Acima de 200 Meses       9093
dtype: int64

In [706]:
#ordenando a coluna Faixa_Prazo_Restante
categorias = ["Até 60 Meses", "De 61 até 120 Meses", "De 121 até 200 Meses", "Acima de 200 Meses"]
df["Faixa_Prazo_Restante"] = df["Faixa_Prazo_Restante"].cat.set_categories(categorias, ordered = True)

print(df["Faixa_Prazo_Restante"].cat.ordered)
print(df["Faixa_Prazo_Restante"].unique())

True
['De 121 até 200 Meses', 'Até 60 Meses', 'De 61 até 120 Meses', 'Acima de 200 Meses']
Categories (4, object): ['Até 60 Meses' < 'De 61 até 120 Meses' < 'De 121 até 200 Meses' < 'Acima de 200 Meses']


In [707]:
#mapeando cada estado para uma regiao

norte = ["AC", "AP", "AM", "PA", "RO", "RR", "TO"]
nordeste = ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]
sudeste = ["ES", "MG", "RJ", "SP"]
sul = ["PR", "RS", "SC"]
centro_oeste = ["DF", "GO", "MT", "MS"]

regiao_estados = {}

for estado in norte:
    regiao_estados[estado] = "Norte"
    
for estado in nordeste:
    regiao_estados[estado] = "Nordeste"    
    
for estado in sudeste:
    regiao_estados[estado] = "Sudeste"   
    
for estado in sul:
    regiao_estados[estado] = "Sul"  
    
for estado in centro_oeste:
    regiao_estados[estado] = "Centro_Oeste"         
    
def get_region(estado):
    return regiao_estados.get(estado, "Nenhum")

df["Regiao_UF_Cliente"] = df["UF_Cliente"].apply(get_region)
df["Regiao_UF_Cliente"].sample(7)

58655         Sudeste
66465         Sudeste
10649        Nordeste
22001    Centro_Oeste
23891    Centro_Oeste
13771             Sul
52219        Nordeste
Name: Regiao_UF_Cliente, dtype: object

## Visualização com gráficos

In [708]:
# #grafico de barras mostrando as faixas de saldo devedor por sexo

# df_count = df.groupby(["Faixa_Saldo_Devedor", "Sexo"]).size().reset_index(name="Count")

# ordenando os dados
# df_count = df_count.sort_values(by="Count", ascending=False)

# fig = px.bar(df_count, x="Faixa_Saldo_Devedor", y="Count", color="Sexo", text_auto=True, title="Número de Clientes por Saldo Devedor e por Sexo",template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/ClientesSaldoSexo.html", full_html=False, include_plotlyjs='cdn')

In [709]:
#valor de renda por escolaridade sem a informacao "Nenhum" 

# df_count = df.groupby(["Escolaridade", "Faixa_Salarial"]).size().reset_index(name="Count")

# ordenando os dados
# df_count = df_count.sort_values(by="Count", ascending=False)

# filtro = df_count[df_count["Escolaridade"] != "Nenhum"]
# fig = px.bar(filtro, barmode = "group",  x="Faixa_Salarial", y="Count", color="Escolaridade", text_auto=True, title="Faixa de Salário por Escolaridade (Sem a Escolaridade = Nenhum)",template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/EscolaridadeSalario.html", full_html=False, include_plotlyjs='cdn')

In [710]:
#grafico sobre a quantidade de parcelas pagas, pagas em dia e pagas em atraso

# total_em_dia = df["QT_Total_Parcelas_Pagas_EmDia"].sum()
# total_em_atraso = df["QT_Total_Parcelas_Pagas_EmAtraso"].sum()

# filtro = pd.DataFrame({
#     "Status": ["Em Dia", "Em Atraso"],
#     "Count": [total_em_dia, total_em_atraso]
# })

# fig = px.pie(filtro, names="Status", values="Count", title="Distribuição de Pagamentos em Dia x Em Atraso",template="plotly_dark")

# # porcentagem
# fig.update_traces(textinfo="percent+label", textfont_size=15, marker=dict(line=dict(color='#000000', width=2)))

# # customizacao do hover
# fig.update_traces(hoverinfo="label+percent+value")

# # ajustando o layout
# fig.update_layout(title_font_size=20, title_x=0.5, legend_title_text="Status do Pagamento")

# fig.show()
# fig.write_html("html/ParcelasPagas.html", full_html=False, include_plotlyjs='cdn')

In [711]:
#grafico das regioes

#selecionando o numero de clientes por estado

# df_count = df["UF_Cliente"].value_counts().reset_index()
# df_count.columns = ["UF_Cliente", "Quantidade"]

# fig = px.choropleth(df_count, 
#                     geojson="https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson", 
#                     locations="UF_Cliente", 
#                     featureidkey="properties.sigla", 
#                     color="Quantidade",
#                     title="Clientes por Estados",
#                     scope="south america",
#                     )

# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(
#     margin={"r":0,"t":50,"l":0,"b":0},
#     title_text="Número de clientes por estado",
#     template="plotly_dark"  
# )

# fig.show()
# fig.write_html("html/ClientesEstado.html", full_html=False, include_plotlyjs='cdn')

In [712]:
#grafico de estado civil x media de parcelas em atraso

# df_mean = df.groupby("Estado_Civil")["QT_Parcelas_Atraso"].mean().reset_index()
# df_mean.columns = ["Estado_Civil", "media_QT_Parcelas_Atraso"]

# fig = px.pie(df_mean, 
#             names="Estado_Civil", 
#             values="media_QT_Parcelas_Atraso", 
#             title="Média da Quantidade de Parcelas em Atraso por Estado Civil",
#             color="media_QT_Parcelas_Atraso",
#             template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/EstadoCivilParcelas.html", full_html=False, include_plotlyjs='cdn')

In [713]:
#sunburst da quantidade de Parcelas em Atraso por Estado e Estado Civil

# fig = px.sunburst(df, 
#                 path=["Estado_Civil", "UF_Cliente"], 
#                 values="QT_Parcelas_Atraso",  # Replace with the actual column name for the value
#                 title="Quantidade de Parcelas em Atraso por Estado e Estado Civil",
#                 labels={"Estado_Civil": "Estado Civil", "UF_Cliente": "UF Cliente"},
#                 width=1000,
#                 height=800,
#                 template="plotly_dark")  

# fig.show()
# fig.write_html("html/EstadoCivilAtraso.html", full_html=False, include_plotlyjs='cdn')

In [714]:
#grafico sobre as faixas de prazo de emprestimo

# df_count = df["Faixa_Prazo_Emprestimo"].value_counts().reset_index()
# df_count.columns = ["Faixa_Prazo_Emprestimo", "Count"]

# #ordenando os dados
# df_count = df_count.sort_values(by="Count", ascending=False)

# fig = px.bar(df_count, x="Faixa_Prazo_Emprestimo", y="Count", color="Faixa_Prazo_Emprestimo", title="Faixas de Prazo de Empréstimo", template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/PrazoEmprestimo.html", full_html=False, include_plotlyjs='cdn')

In [715]:
#grafico sobre as faixas de prazo restante

# df_count = df["Faixa_Prazo_Restante"].value_counts().reset_index()
# df_count.columns = ["Faixa_Prazo_Restante", "Count"]

# #ordenando os dados
# df_count = df_count.sort_values(by="Count", ascending=False)

# fig = px.bar(df_count, x="Faixa_Prazo_Restante", y="Count", color="Faixa_Prazo_Restante", title="Faixas de Prazo Restante", text_auto=True,template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/PrazoRestante.html", full_html=False, include_plotlyjs='cdn')

In [716]:
#grafico sobre as faixas etaria

# df_count = df["Faixa_Etaria"].value_counts().reset_index()
# df_count.columns = ["Faixa_Etaria", "Count"]

# #ordenando os dados
# df_count = df_count.sort_values(by="Count", ascending=False)

# fig = px.bar(df_count, x="Faixa_Etaria", y="Count", color="Faixa_Etaria", title="Faixas Etária", text_auto=True, template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()
# fig.write_html("html/FaixaEtaria.html", full_html=False, include_plotlyjs='cdn')

## Novo Tratamento dos dados

In [717]:
df.columns

Index(['Contrato', 'Idade', 'Sexo', 'Valor_Renda', 'UF_Cliente', 'Perc_Juros',
       'Prazo_Emprestimo', 'Data_Contratacao', 'Prazo_Restante',
       'VL_Emprestimo', 'VL_Emprestimo_ComJuros', 'QT_Total_Parcelas_Pagas',
       'QT_Total_Parcelas_Pagas_EmDia', 'QT_Total_Parcelas_Pagas_EmAtraso',
       'Qt_Renegociacao', 'Estado_Civil', 'Escolaridade', 'Possui_Patrimonio',
       'VL_Patrimonio', 'QT_Parcelas_Atraso', 'QT_Dias_Atraso',
       'Saldo_Devedor', 'Total_Pago', 'Possivel_Fraude', 'Faixa_Etaria',
       'Faixa_Salarial', 'Faixa_Dias_Atraso', 'Faixa_Total_Pago',
       'Faixa_Saldo_Devedor', 'Faixa_Prazo_Emprestimo', 'Faixa_Prazo_Restante',
       'Regiao_UF_Cliente'],
      dtype='object')

In [718]:
#Excluindo colunas que nao farao parte do nosso modelo/analise

#Contrato -> identificacao do cliente, nao tem relevancia aqui
#Iade -> Substituido por Faixa Etaria
#Valor Renda, Prazo Emprestimo, Dias em Atraso, Prazo Restante -> Substituido por faixas criadas
#Data Contratacao -> Nao é uma Time Series, nao usaremos data aqui
#Valor Patrimonio -> Dados incompletos
#Possui Patrimonio -> Dados incompletos
#Escolaridade -> Muitos dados com "Nenhum"


colunas = ["Sexo", "Regiao_UF_Cliente", "Perc_Juros", "VL_Emprestimo", "VL_Emprestimo_ComJuros", "QT_Total_Parcelas_Pagas", "QT_Total_Parcelas_Pagas_EmDia", 
            "QT_Total_Parcelas_Pagas_EmAtraso", "Qt_Renegociacao", "Estado_Civil", "QT_Parcelas_Atraso", "Faixa_Etaria", 
            "Faixa_Salarial", "Faixa_Dias_Atraso", "Faixa_Total_Pago", "Faixa_Saldo_Devedor", "Faixa_Prazo_Emprestimo", "Faixa_Prazo_Restante", "Possivel_Fraude"]

df_tratado = pd.DataFrame(df, columns=colunas)

In [719]:
df_tratado.shape

(80143, 19)

In [720]:
df_tratado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Sexo                              80143 non-null  category
 1   Regiao_UF_Cliente                 80143 non-null  object  
 2   Perc_Juros                        80143 non-null  float32 
 3   VL_Emprestimo                     80143 non-null  float32 
 4   VL_Emprestimo_ComJuros            80143 non-null  float32 
 5   QT_Total_Parcelas_Pagas           80143 non-null  int8    
 6   QT_Total_Parcelas_Pagas_EmDia     80143 non-null  int8    
 7   QT_Total_Parcelas_Pagas_EmAtraso  80143 non-null  int8    
 8   Qt_Renegociacao                   80143 non-null  int8    
 9   Estado_Civil                      80143 non-null  category
 10  QT_Parcelas_Atraso                80143 non-null  int8    
 11  Faixa_Etaria                      80143 non-null  cate

In [721]:
#verificando se tem valores ausentes
df_tratado.isnull().sum()

Sexo                                0
Regiao_UF_Cliente                   0
Perc_Juros                          0
VL_Emprestimo                       0
VL_Emprestimo_ComJuros              0
QT_Total_Parcelas_Pagas             0
QT_Total_Parcelas_Pagas_EmDia       0
QT_Total_Parcelas_Pagas_EmAtraso    0
Qt_Renegociacao                     0
Estado_Civil                        0
QT_Parcelas_Atraso                  0
Faixa_Etaria                        0
Faixa_Salarial                      0
Faixa_Dias_Atraso                   0
Faixa_Total_Pago                    0
Faixa_Saldo_Devedor                 0
Faixa_Prazo_Emprestimo              0
Faixa_Prazo_Restante                0
Possivel_Fraude                     0
dtype: int64

## Análise Exploratória em Variáveis Categóricas 

#### Vamos observar as variáves categóricas e como elas se relacionam com a Variável Alvo (Possivel_Fraude)

In [722]:
df_tratado.describe()

,Perc_Juros,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,QT_Parcelas_Atraso
count,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00
mean,19.49,72847.77,83774.95,10.71,5.24,2.34,1.30,10.65
std,3.56,87407.94,100519.12,9.31,6.21,3.97,1.27,10.43
min,4.49,3000.00,3450.00,0.00,0.00,0.00,0.00,0.00
25%,18.00,15000.00,17250.00,2.00,1.00,0.00,0.00,0.00
50%,20.00,40000.00,46000.00,8.00,2.00,1.00,1.00,9.00
75%,21.00,100000.00,115000.00,18.00,7.00,3.00,2.00,20.00
max,28.00,500000.00,575000.00,66.00,66.00,31.00,38.00,34.00


In [723]:
#mapeamento das cores
color_map = {
    "Sim": "#EF553B",
    "Nao": "#636EFA"
}

Observamos que existe um tendência de quando o Total Pago for maior do que 60K, possivelmente não existe fraude.

Agora, vamos fazer uma série de histogramas e comparar com a variável Possivel_Fraude 

In [724]:
# fig = px.histogram(df_tratado, x="Sexo", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/SexoFraude.html", full_html=False, include_plotlyjs='cdn')

In [725]:
# df_tratado = df_tratado.sort_values(by="Faixa_Etaria", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Etaria", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/Faixa_EtariaFraude.html", full_html=False, include_plotlyjs='cdn')

In [726]:
# fig = px.histogram(df_tratado, x="Regiao_UF_Cliente", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/RegiaoFraude.html", full_html=False, include_plotlyjs='cdn')

In [727]:
# df_tratado = df_tratado.sort_values(by="Faixa_Salarial", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Salarial", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/SalarioFraude.html", full_html=False, include_plotlyjs='cdn')

In [728]:
# df_tratado = df_tratado.sort_values(by="QT_Total_Parcelas_Pagas", ascending=True)

# fig = px.histogram(df_tratado, x="QT_Total_Parcelas_Pagas", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/ParcelasPagasFraude.html", full_html=False, include_plotlyjs='cdn')

In [729]:
# df_tratado = df_tratado.sort_values(by="QT_Total_Parcelas_Pagas_EmDia", ascending=True)

# fig = px.histogram(df_tratado, x="QT_Total_Parcelas_Pagas_EmDia", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/PagasEmDiaFraude.html", full_html=False, include_plotlyjs='cdn')

In [730]:
# df_tratado = df_tratado.sort_values(by="QT_Total_Parcelas_Pagas_EmAtraso", ascending=True)

# fig = px.histogram(df_tratado, x="QT_Total_Parcelas_Pagas_EmAtraso", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/PagasEmAtrasoFraude.html", full_html=False, include_plotlyjs='cdn')

In [731]:
# df_tratado = df_tratado.sort_values(by="QT_Parcelas_Atraso", ascending=True)

# fig = px.histogram(df_tratado, x="QT_Parcelas_Atraso", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/ParcelasAtrasoFraude.html", full_html=False, include_plotlyjs='cdn')

In [732]:
# df_tratado = df_tratado.sort_values(by="Faixa_Total_Pago", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Total_Pago", color="Possivel_Fraude", barmode="group", color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/TotalPagoFraude.html", full_html=False, include_plotlyjs='cdn')

In [733]:
# df_tratado = df_tratado.sort_values(by="Qt_Renegociacao", ascending=True)

# df_filtro = df_tratado[df_tratado["Qt_Renegociacao"] < 15]

# fig = px.histogram(df_filtro, x="Qt_Renegociacao", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/RenegociacaoFraude.html", full_html=False, include_plotlyjs='cdn')

In [734]:
# df_tratado = df_tratado.sort_values(by="Faixa_Prazo_Restante", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Prazo_Restante", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/PrazoFraude.html", full_html=False, include_plotlyjs='cdn')

In [735]:
# df_tratado = df_tratado.sort_values(by="Faixa_Dias_Atraso", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Dias_Atraso", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/DiasAtrasoFraude.html", full_html=False, include_plotlyjs='cdn')

In [736]:
# df_tratado = df_tratado.sort_values(by="Faixa_Saldo_Devedor", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Saldo_Devedor", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/DevedorFraude.html", full_html=False, include_plotlyjs='cdn')

In [737]:
# df_tratado = df_tratado.sort_values(by="Faixa_Prazo_Emprestimo", ascending=True)

# fig = px.histogram(df_tratado, x="Faixa_Prazo_Emprestimo", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/EmprestimoFraude.html", full_html=False, include_plotlyjs='cdn')

In [738]:
# df_tratado = df_tratado.sort_values(by="Estado_Civil", ascending=True)

# fig = px.histogram(df_tratado, x="Estado_Civil", color="Possivel_Fraude", barmode="group", text_auto=True, color_discrete_map=color_map, template="plotly_dark")
# fig.show()
# fig.write_html("html/EstadoCivilFraude.html", full_html=False, include_plotlyjs='cdn')

## Análise Exploratória em Variáveis Numéricas, OneHotEnconding e novo Tratamento dos Dados

#### Verificar média, mediana, moda, desvio padrão, correlação, outliers, distribuição dos dados
#### Preparar os dados para os modelos de Machine Learning

In [739]:
#construir um df apenas para as variaveis numericas
df_numerico = df_tratado.select_dtypes(include=['number'])
df_numerico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perc_Juros                        80143 non-null  float32
 1   VL_Emprestimo                     80143 non-null  float32
 2   VL_Emprestimo_ComJuros            80143 non-null  float32
 3   QT_Total_Parcelas_Pagas           80143 non-null  int8   
 4   QT_Total_Parcelas_Pagas_EmDia     80143 non-null  int8   
 5   QT_Total_Parcelas_Pagas_EmAtraso  80143 non-null  int8   
 6   Qt_Renegociacao                   80143 non-null  int8   
 7   QT_Parcelas_Atraso                80143 non-null  int8   
dtypes: float32(3), int8(5)
memory usage: 1.3 MB


In [794]:
#verificando a correlacao das variaveis numericas
# fig = px.imshow(df_numerico.corr(), aspect="auto", text_auto=True, template="plotly_dark")
# fig.show()
# fig.write_html("html/Correlacao.html", full_html=False, include_plotlyjs='cdn')

In [741]:
df_numerico[["VL_Emprestimo", "VL_Emprestimo_ComJuros"]].sample(5)

,VL_Emprestimo,VL_Emprestimo_ComJuros
77265,200000.00,230000.00
51989,75000.00,86250.00
71211,70000.00,80500.00
3516,320000.00,368000.00
5975,250000.00,287500.00


Observamos que VL_Emprestimo e VL_Emprestimo_ComJuros possuem alta cardinalidade positiva entre si, e que QT_Parcelas_Atraso e QT_Total_Parcelas_Pagas possuem uma alta cardinalidade negativa entre si, o que faz sentido já que essas informações dependem das outras ou são contrárias. Isso será resolvido na etapa de padronização e normalização de dados.

In [742]:
#boxplots para verificar outliers, mediana, valores maximos, minimos e quartils

#criar automaticamente os plots
# for column in df_numerico.columns:
#     fig = px.box(df_numerico, y=column, title=f'Boxplot para {column}')
#     fig.update_layout(
#     width=1200,  
#     height=800   
#     )
#     fig.show()

In [743]:
#carregando as variaveis nao numericas

colunas_numericas = df_tratado.select_dtypes(include=['number']).columns
coluna_alvo = df_tratado[["Possivel_Fraude"]]

df_categorica = df_tratado.drop(columns=colunas_numericas).drop(columns=coluna_alvo)
df_categorica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Sexo                    80143 non-null  category
 1   Regiao_UF_Cliente       80143 non-null  object  
 2   Estado_Civil            80143 non-null  category
 3   Faixa_Etaria            80143 non-null  category
 4   Faixa_Salarial          80143 non-null  category
 5   Faixa_Dias_Atraso       80143 non-null  category
 6   Faixa_Total_Pago        80143 non-null  category
 7   Faixa_Saldo_Devedor     80143 non-null  category
 8   Faixa_Prazo_Emprestimo  80143 non-null  category
 9   Faixa_Prazo_Restante    80143 non-null  category
dtypes: category(9), object(1)
memory usage: 1.3+ MB


In [744]:
#OrdinalEnconding, vamos separar os dados para usar o Ordinal Encoding para colunas com uma ordem especifica (Faixa_Etaria),
#e OneHotEncoding para aquelas que nao tem ordem(Sexo)

ore = OrdinalEncoder()
ohe = OneHotEncoder(sparse_output=False)

colunas_ordinal_encoding = ["Faixa_Etaria", "Faixa_Salarial", "Faixa_Dias_Atraso", "Faixa_Total_Pago", "Faixa_Saldo_Devedor", "Faixa_Prazo_Emprestimo", "Faixa_Prazo_Restante"]
colunas_onehot_encoding = ["Sexo", "Regiao_UF_Cliente", "Estado_Civil"]

ore_ft = ore.fit_transform(df_categorica[colunas_ordinal_encoding])
ore_df = pd.DataFrame(ore_ft, columns=colunas_ordinal_encoding)

ohe_ft = ohe.fit_transform(df_categorica[colunas_onehot_encoding])
ohe_df = pd.DataFrame(ohe_ft, columns=ohe.get_feature_names_out(colunas_onehot_encoding))

df_categorico_encoded = pd.concat([ore_df, ohe_df], axis=1)

df_transformado = pd.concat([df_categorico_encoded, df_numerico, df_tratado[["Possivel_Fraude"]]], axis=1)

df_transformado.sample(5)

,Faixa_Etaria,Faixa_Salarial,Faixa_Dias_Atraso,Faixa_Total_Pago,Faixa_Saldo_Devedor,Faixa_Prazo_Emprestimo,Faixa_Prazo_Restante,Sexo_F,Sexo_M,Regiao_UF_Cliente_Centro_Oeste,Regiao_UF_Cliente_Nordeste,Regiao_UF_Cliente_Norte,Regiao_UF_Cliente_Sudeste,Regiao_UF_Cliente_Sul,Estado_Civil_CASADO(A),Estado_Civil_DIVORCIADO,Estado_Civil_OUTRO,Estado_Civil_SOLTEIRO(A),Estado_Civil_VIÚVO(A),Perc_Juros,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,QT_Parcelas_Atraso,Possivel_Fraude
32468,1.00,2.00,12.00,2.00,8.00,1.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,21.00,15000.00,17250.00,34,31,0,0,0,Nao
57240,4.00,2.00,12.00,7.00,4.00,3.00,3.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,18.00,80000.00,92000.00,12,5,4,3,0,Nao
30361,0.00,2.00,12.00,8.00,4.00,1.00,3.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,23.00,20000.00,23000.00,12,3,3,1,0,Nao
75209,2.00,5.00,12.00,8.00,2.00,3.00,3.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,25.00,18000.00,20700.00,20,17,0,0,0,Nao
34211,1.00,2.00,12.00,8.00,4.00,3.00,3.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,23.00,25000.00,28750.00,12,11,0,0,0,Nao


In [745]:
#verificando o numero de colunas depois da transformacao
df_transformado.shape

(80143, 28)

In [746]:
#verificando se existem valores nulos
df_transformado.isnull().sum()

Faixa_Etaria                        0
Faixa_Salarial                      0
Faixa_Dias_Atraso                   0
Faixa_Total_Pago                    0
Faixa_Saldo_Devedor                 0
Faixa_Prazo_Emprestimo              0
Faixa_Prazo_Restante                0
Sexo_F                              0
Sexo_M                              0
Regiao_UF_Cliente_Centro_Oeste      0
Regiao_UF_Cliente_Nordeste          0
Regiao_UF_Cliente_Norte             0
Regiao_UF_Cliente_Sudeste           0
Regiao_UF_Cliente_Sul               0
Estado_Civil_CASADO(A)              0
Estado_Civil_DIVORCIADO             0
Estado_Civil_OUTRO                  0
Estado_Civil_SOLTEIRO(A)            0
Estado_Civil_VIÚVO(A)               0
Perc_Juros                          0
VL_Emprestimo                       0
VL_Emprestimo_ComJuros              0
QT_Total_Parcelas_Pagas             0
QT_Total_Parcelas_Pagas_EmDia       0
QT_Total_Parcelas_Pagas_EmAtraso    0
Qt_Renegociacao                     0
QT_Parcelas_

In [747]:
#otimizando novamente as colunas numericas
colunas_float = df_transformado.select_dtypes(include="float64").columns

df_transformado[colunas_float] = df_transformado[colunas_float].apply(pd.to_numeric, downcast="float")

In [748]:
df_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Faixa_Etaria                      80143 non-null  float32 
 1   Faixa_Salarial                    80143 non-null  float32 
 2   Faixa_Dias_Atraso                 80143 non-null  float32 
 3   Faixa_Total_Pago                  80143 non-null  float32 
 4   Faixa_Saldo_Devedor               80143 non-null  float32 
 5   Faixa_Prazo_Emprestimo            80143 non-null  float32 
 6   Faixa_Prazo_Restante              80143 non-null  float32 
 7   Sexo_F                            80143 non-null  float32 
 8   Sexo_M                            80143 non-null  float32 
 9   Regiao_UF_Cliente_Centro_Oeste    80143 non-null  float32 
 10  Regiao_UF_Cliente_Nordeste        80143 non-null  float32 
 11  Regiao_UF_Cliente_Norte           80143 non-null  floa

## Separando as Variáveis Preditoras da Váriavel Alvo e Balanceamento da Variável Alvo

In [749]:
#separando as variaveis preditoras da variavel alvo

var_preditoras = df_transformado.drop("Possivel_Fraude", axis = 1)
print("Variáveis Preditoras: ",var_preditoras.shape)

var_target = df_transformado[["Possivel_Fraude"]]
print("Variável Alvo: ", var_target.shape)

Variáveis Preditoras:  (80143, 27)
Variável Alvo:  (80143, 1)


In [750]:
#visualizando os dados
var_preditoras.sample(5)

,Faixa_Etaria,Faixa_Salarial,Faixa_Dias_Atraso,Faixa_Total_Pago,Faixa_Saldo_Devedor,Faixa_Prazo_Emprestimo,Faixa_Prazo_Restante,Sexo_F,Sexo_M,Regiao_UF_Cliente_Centro_Oeste,Regiao_UF_Cliente_Nordeste,Regiao_UF_Cliente_Norte,Regiao_UF_Cliente_Sudeste,Regiao_UF_Cliente_Sul,Estado_Civil_CASADO(A),Estado_Civil_DIVORCIADO,Estado_Civil_OUTRO,Estado_Civil_SOLTEIRO(A),Estado_Civil_VIÚVO(A),Perc_Juros,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,QT_Parcelas_Atraso
18063,1.00,2.00,6.00,6.00,4.00,3.00,3.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,20.00,40000.00,46000.00,7,6,1,0,10
71187,4.00,2.00,12.00,2.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,18.00,9000.00,10350.00,5,1,4,1,0
41844,2.00,2.00,12.00,2.00,4.00,3.00,3.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,17.88,30000.00,34500.00,25,25,0,1,0
58739,3.00,5.00,7.00,4.00,2.00,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,18.00,35000.00,40250.00,22,1,21,3,2
64321,1.00,2.00,12.00,6.00,7.00,3.00,3.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,18.00,60000.00,69000.00,5,1,3,1,20


In [751]:
#visualizando os dados
var_target.sample(5)

,Possivel_Fraude
41451,Sim
24022,Nao
6237,Nao
79251,Sim
46038,Sim


In [752]:
#quantidade de dados da variavel alvo
df_transformado[["Possivel_Fraude"]].value_counts()

Possivel_Fraude
Sim                44511
Nao                35632
Name: count, dtype: int64

In [753]:
#balancear a variavel alvo para nao criar vies no modelo
balanceador = SMOTE()

var_preditoras_b, var_target_b = balanceador.fit_resample(var_preditoras, var_target)


In [754]:
#grafico que mostra Possíveis Fraude x Não Fraudes
# fig = px.bar(var_target_b, title="Possíveis Fraude x Não Fraudes", color="value", template="plotly_dark")
# fig.update_traces(dict(marker_line_width=0))
# fig.show()

## Normalização/Padronização de Dados

#### Iremos testar esses métodos para ver qual se sai melhor nos modelos, também usaremos o RobustScaler por conta dos outliers em algumas colunas

In [755]:
#separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(var_preditoras_b, var_target_b, test_size=0.3, random_state=10)

In [756]:
print("Dados de Treino: ", X_train.shape)

Dados de Treino:  (62315, 27)


In [757]:
#normalizacao
normalizador = MinMaxScaler()
X_treino_normalizado = normalizador.fit_transform(X_train)
X_teste_normalizado = normalizador.transform(X_test)

In [758]:
#vendo o tamanho dos dados
X_treino_normalizado.shape

(62315, 27)

In [759]:
#observando os dados
X_treino_normalizado

array([[0.16666667, 0.5       , 1.        , ..., 0.16129032, 0.07692308,
        0.        ],
       [0.33333334, 0.2       , 1.        , ..., 0.03225806, 0.07692308,
        0.61764705],
       [0.5       , 0.2       , 0.75      , ..., 0.        , 0.        ,
        0.3529412 ],
       ...,
       [0.5       , 0.5       , 1.        , ..., 0.        , 0.07692308,
        0.5882353 ],
       [0.33333334, 0.5       , 0.4166667 , ..., 0.2580645 , 0.11538462,
        0.05882353],
       [0.38463223, 0.5       , 0.        , ..., 0.48387095, 0.07692308,
        0.02941176]], dtype=float32)

In [760]:
#padronizacao
padronizador = StandardScaler()
X_treino_padronizado = padronizador.fit_transform(X_train)
X_teste_padronizado = padronizador.transform(X_test)

In [761]:
#vendo o tamanho dos dados
X_treino_padronizado.shape

(62315, 27)

In [762]:
#observando os dados
X_treino_padronizado

array([[-1.0679319 ,  0.56602144,  0.48162186, ...,  0.6216999 ,
         0.5473783 , -0.93120193],
       [-0.27132317, -0.75408274,  0.48162186, ..., -0.3584146 ,
         0.5473783 ,  1.0946137 ],
       [ 0.52528554, -0.75408274, -0.3869191 , ..., -0.60344315,
        -1.040568  ,  0.22640699],
       ...,
       [ 0.52528554,  0.56602144,  0.48162186, ..., -0.60344315,
         0.5473783 ,  0.99814624],
       [-0.27132317,  0.56602144, -1.5449739 , ...,  1.3567858 ,
         1.3413515 , -0.73826706],
       [-0.02613229,  0.56602144, -2.992542  , ...,  3.0719862 ,
         0.5473783 , -0.8347345 ]], dtype=float32)

In [763]:
#RobustScaler, para dados com outliers
robusto = RobustScaler()
X_treino_robusto = robusto.fit_transform(X_train)
X_teste_robusto = robusto.transform(X_test)

In [764]:
#vendo o tamanho dos dados
X_treino_robusto.shape

(62315, 27)

In [765]:
#observando os dados
X_treino_robusto

array([[ -0.5       ,   1.        ,   0.        , ...,   1.3333334 ,
          0.5       ,  -0.31578946],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.5       ,   0.7894737 ],
       [  0.5       ,   0.        ,  -3.        , ...,  -0.33333334,
         -0.5       ,   0.31578946],
       ...,
       [  0.5       ,   1.        ,   0.        , ...,  -0.33333334,
          0.5       ,   0.7368421 ],
       [  0.        ,   1.        ,  -7.        , ...,   2.3333333 ,
          1.        ,  -0.21052632],
       [  0.15389669,   1.        , -12.        , ...,   4.6666665 ,
          0.5       ,  -0.2631579 ]], dtype=float32)

## Modelos de Machine Learning
#### Criando, Treinando e Avaliando
#### Arrumando Hyperparâmetros

In [766]:
#usando o RandomForest
rf = RandomForestClassifier()

# Valores para o grid de hiperparametros
n_estimators = np.array([100,200,300])
max_depth = np.array([10,20])
criterion = np.array(["gini", "entropy"])
max_features = np.array(["sqrt", "log2", None])
min_samples_split = np.array([1,2,5])
min_samples_leaf = np.array([1,2,3])

# Grid de hiperparâmetros
grid_parametros = dict(n_estimators = n_estimators,
                    max_depth = max_depth,
                    criterion = criterion,
                    max_features = max_features,
                    min_samples_split = min_samples_split,
                    min_samples_leaf = min_samples_leaf)


# Criando o modelo com o Grid de Hiperparametros
randomForest = GridSearchCV(rf, grid_parametros, cv = 3, n_jobs = 8)

In [767]:
# Treinando o modelo normalizado
# inicio = time.time()
# randomForest.fit(X_treino_normalizado, y_train)
# fim = time.time()

In [768]:
# Obtendo e visualizando os parametros treinados
#treino_rf_n = pd.DataFrame(randomForest.cv_results_)

In [769]:
# Acurácia em Treino normalizado
# print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Numero de treinamentos realizados: ", treino_rf_n.shape[0])

In [770]:
# Treinando o modelo padronizado
# inicio = time.time()
# randomForest.fit(X_treino_padronizado, y_train)
# fim = time.time()

In [771]:
#treino_rf_p = pd.DataFrame(randomForest.cv_results_)

In [772]:
# Acurácia em Treino normalizado
# print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Numero de treinamentos realizados: ", treino_rf_p.shape[0])

In [773]:
# #Treinando o modelo robusto
# inicio = time.time()
# randomForest.fit(X_treino_robusto, y_train)
# fim = time.time()

In [774]:
# treino_rf_r = pd.DataFrame(randomForest.cv_results_)

In [775]:
# Acurácia em Treino normalizado
# print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Numero de treinamentos realizados: ", treino_rf_r.shape[0])

No geral, os três modelos tiveram resultados parecidos, vamos usar o RobustScaler com os Hiperparâmetros ideais

In [776]:
# Criando o classificador com Random Forest
clf = RandomForestClassifier(n_estimators  = 200, criterion = 'entropy', max_depth = 20, 
                            max_features = 'log2', min_samples_leaf = 2, min_samples_split = 5, n_jobs = 8)

# Construção do modelo
clf = clf.fit(X_treino_robusto, y_train)

In [777]:
#vendo a importancia das variaveis no modelo
# importancia = pd.Series(data=clf.feature_importances_, index=var_preditoras_b.columns)
# importancia = importancia.sort_values(ascending=True)
# fig = px.bar(x=importancia, y=importancia.index, orientation="h", title="Importância das colunas para o modelo")
# fig.show()

In [778]:
# Visualizando o percentual de importancia de cada variável
# importancia.sort_values(ascending = False)

In [779]:
#simulando os dados de treino
resultado = clf.score(X_treino_robusto, y_train)
print(resultado)

0.9981866324319987


In [780]:
#simulando os dados de teste (valor sempre menor do que os dados de treino)
resultado = clf.score(X_teste_robusto, y_test)
print(resultado)

0.992473883251582


In [781]:
# Dicionário de métricas e metadados para comparar com outros modelos
# modelo_rf = {"Melhores Hiperparametros":randomForest.best_params_,
#             "Numero de Modelos Treinados": treino_rf_r.shape[0],
#             "Melhor Score": str(round(randomForest.best_score_ * 100,2))+"%"}

# display(modelo_rf)

In [782]:
#testando o modelo SVM
svm = SVC(probability=False)

# Valores para o grid de hiperparametros
grid_parametros = {"C": [0.1,1,10,100],
                    "gamma": [1,0.1,0.01,0.001],
                    #"kernel": ["poly","rbf","sigmoid","linear"], -> demoram bastante
                    "kernel": ["linear"], # -> mais rapido
                    "degree" : [2,3,4,],
                    "coef0" : [0.5,1],
                    #"decision_function_shape":["ovo", "ovr"], -> demoram bastante
                    "max_iter": [-1, 1]}


supportvectorm = GridSearchCV(svm, grid_parametros, n_jobs = 8)

In [783]:
# Treinando os modelos com os dados robustos
# inicio = time.time()
# supportvectorm.fit(X_treino_robusto, y_train)
# fim = time.time()

In [784]:
# Obtendo e visualizando os parametros treinados
# treino_svm = pd.DataFrame(supportvectorm.cv_results_)

In [785]:
# Acurácia em Treino
# print(f"Acurácia em Treinamento: {supportvectorm.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {supportvectorm.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo (s): ", round(fim - inicio,2))
# print("")
# print("Numero de treinamentos realizados: ", treino_svm.shape[0])

In [786]:
# Dicionário de métricas e metadados
# modelo_svm = {"Melhores Hiperparametros":supportvectorm.best_params_,
#             "Numero de Modelos Treinados": treino_svm.shape[0],
#             "Melhor Score": str(round(supportvectorm.best_score_ * 100,2))+"%"}

Verificamos que o modelo SVM demorou muito mais tempo para ser executado e obteve uma Acurácia menor

In [787]:
#testando o modelo KNN
knn = KNeighborsClassifier()

# Valores para o grid de hiperparametros
n_neighbors = np.array([3,5,7,9])
algorithm = np.array(["auto", "ball_tree", "kd_tree", "brute"])
leaf_size = np.array([30,31,32])
metric =  np.array(["minkowski","euclidean"])


# Grid de hiperparâmetros
grid_parametros = dict(n_neighbors = n_neighbors,
                    algorithm = algorithm,
                    leaf_size = leaf_size,
                    metric = metric)

knn = GridSearchCV(knn, grid_parametros, n_jobs = 8)

In [788]:
# Treinando os modelos
# inicio = time.time()
# knn.fit(X_treino_robusto, y_train)
# fim = time.time()

In [789]:
# Obtendo e visualizando os parametros treinados
# treinos_knn = pd.DataFrame(knn.cv_results_)

In [790]:
# Acurácia em Treino
# print(f"Acurácia em Treinamento: {knn.best_score_ :.2%}")
# print("")
# print(f"Hiperparâmetros Ideais: {knn.best_params_}")
# print("")
# print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
# print("")
# print("Numero de treinamentos realizados: ", treinos_knn.shape[0])

Observamos que o modelo KNN obteve a menor Acurácia entre os três testados, como o primeiro modelo apresentou um resultado bem satisfatório, não iremos treinar nenhum outro modelo.

In [791]:
# Dicionário de métricas e metadados
# modelo_knn = {"Melhores Hiperparametros":knn.best_params_,
#             "Numero de Modelos Treinados": treinos_knn.shape[0],
#             "Melhor Score": str(round(knn.best_score_ * 100,2))+"%"}

In [792]:
# Gerando o DataFrame com todos os valores de todos os modelos treinados
# resumo = pd.DataFrame({'Random Forest':pd.Series(modelo_rf),
#                     'SVM':pd.Series(modelo_svm),
#                     'KNN':pd.Series(modelo_knn)})

In [793]:
# resumo